In [24]:
import pandas as pd
import numpy as np
# !pip install TA_Lib-0.4.17-cp36-cp36m-win_amd64.whl
import talib

In [25]:
df = pd.read_csv('for_cloud.csv')
masks = pd.read_csv('constituents_csv.csv')
df.head()

,ticker,Date,Adj Close,Close,High,Low,Open,Volume,Name,Sector
0,AAL,2008-12-31,7.405228,7.73,7.87,7.48,7.48,4194100.0,American Airlines Group,Industrials
1,AAL,2009-01-02,8.037499,8.39,8.48,7.67,7.73,5167000.0,American Airlines Group,Industrials
2,AAL,2009-01-05,7.980019,8.33,8.39,7.96,8.38,3457100.0,American Airlines Group,Industrials
3,AAL,2009-01-06,8.679349,9.06,9.21,8.13,8.15,5731000.0,American Airlines Group,Industrials
4,AAL,2009-01-07,8.698509,9.08,9.47,8.66,8.66,5468900.0,American Airlines Group,Industrials


In [26]:
print(talib.get_function_groups())

{'Cycle Indicators': ['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE'], 'Math Operators': ['ADD', 'DIV', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'MULT', 'SUB', 'SUM'], 'Math Transform': ['ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH'], 'Momentum Indicators': ['ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR'], 'Overlap Studies': ['BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA'], 'Pattern Recognition': ['CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHO

In [27]:
from ta import *
df.dropna(axis = 0, how = 'any', inplace = True)
df.groupby(['ticker']).size().min()
df.reset_index(inplace = True)
# df['ema_move'] = df.ema_cross - df.ema_cross.shift(1)
# df['rsi_move'] = rsi(df.close,n=18) - rsi(df.close,n=18).shift(1)
# df['bb_diff'] = bollinger_hband(df.close,n=8,ndev=2) - bollinger_lband(df.close, n=8, ndev=2)
# df['ema_cross'] = df.close - ema_fast(df.close,n_fast=4)

In [28]:
# df['bb_high_indicator'] = bollinger_hband_indicator(df["Close"], n=20, ndev=2, fillna=True)
# df['bb_low_indicator'] = bollinger_lband_indicator(df["Close"], n=20, ndev=2, fillna=True)

In [29]:
df.head()

,index,ticker,Date,Adj Close,Close,High,Low,Open,Volume,Name,Sector
0,0,AAL,2008-12-31,7.405228,7.73,7.87,7.48,7.48,4194100.0,American Airlines Group,Industrials
1,1,AAL,2009-01-02,8.037499,8.39,8.48,7.67,7.73,5167000.0,American Airlines Group,Industrials
2,2,AAL,2009-01-05,7.980019,8.33,8.39,7.96,8.38,3457100.0,American Airlines Group,Industrials
3,3,AAL,2009-01-06,8.679349,9.06,9.21,8.13,8.15,5731000.0,American Airlines Group,Industrials
4,4,AAL,2009-01-07,8.698509,9.08,9.47,8.66,8.66,5468900.0,American Airlines Group,Industrials


In [30]:
masks = df.loc[:, 'ticker'].unique()

In [31]:
def not_classical(df, masks):
    
    for mask in masks:
        masky = (df['ticker'] == mask)
        
        #stochastic osc.
        df.loc[masky, '%k_'+str(5)]=(df.loc[masky, 'Close']-df.loc[masky, 'Low'].rolling(5, min_periods=1).min())/(df.loc[masky,'High'].rolling(5, min_periods=1).max()-df.loc[masky, 'Low'].rolling(5, min_periods=1).min())
        df.loc[masky, '%d_'+str(5)+'_'+str(3)]=df.loc[masky, '%k_'+str(5)].rolling(3, min_periods=1).mean()
        df.loc[masky, '%k_'+str(5)+'_'+'%d'+str(3)]=df.loc[masky, '%k_'+str(5)] - df.loc[masky, '%d_'+str(5)+'_'+str(3)] 

        
        df.loc[masky, 'VWAP'] = (df.loc[masky, 'Volume'] * (df.loc[masky, 'High'] + df.loc[masky, 'Low'] + df.loc[masky, 'Close'] ) /3).cumsum() / df.loc[masky,'Volume'].cumsum()
        df.loc[masky, 'MACD_12_26' + 'd'] = macd(df.loc[masky, 'Close'], n_fast=12, n_slow=26, fillna=True)
        df.loc[masky, 'VMA_7MA'] = df.loc[masky, 'Volume'].rolling(window=7).mean()
        df.loc[masky, 'VMA_15MA'] = df.loc[masky, 'Volume'].rolling(window=15).mean()
        df.loc[masky, 'VMA_30MA'] = df.loc[masky, 'Volume'].rolling(window=30).mean()
        
        
        for i in [9, 12, 15]:
            df.loc[masky, 'tenkan_a_'+str(i)]=((df.loc[masky, 'High'].rolling(i, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i, min_periods=1).min())/2)-\
    ((((df.loc[masky, 'High'].rolling(i, min_periods=1).max()+df.loc[masky,'Low'].rolling(i, min_periods=1).min())/2)+\
     ((df.loc[masky, 'High'].rolling(i+15, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i+15, min_periods=1).min())/2))/2).shift(i+15)
    
            df.loc[masky, 'tenkan_b_'+str(i)]=((df.loc[masky, 'High'].rolling(i, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i, min_periods=1).min())/2)-\
    ((df.loc[masky, 'High'].rolling(i+43, min_periods=1).max()+df.loc[masky,'Low'].rolling(i+43, min_periods=1).min())/2).shift(i+15)
    
            df.loc[masky, 'kijun_a_'+str(i)]=((df.loc[masky, 'High'].rolling(i+15, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i+15, min_periods=1).min())/2)-\
    ((((df.loc[masky, 'High'].rolling(i, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i, min_periods=1).min())/2)+\
     ((df.loc[masky, 'High'].rolling(i+15, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i+15, min_periods=1).min())/2))/2).shift(i+15)
    
            df.loc[masky, 'kijun_b_'+str(i)]=((df.loc[masky, 'High'].rolling(i+15, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i+15, min_periods=1).min())/2)-\
    ((((df.loc[masky, 'High'].rolling(i, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i, min_periods=1).min())/2)+\
     ((df.loc[masky, 'High'].rolling(i+15, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i+15, min_periods=1).min())/2))/2).shift(i+15)
    
            df.loc[masky, 'a_b_'+str(i)]=((((df.loc[masky, 'High'].rolling(i, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i, min_periods=1).min())/2)+\
     ((df.loc[masky, 'High'].rolling(i+15, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i+15, min_periods=1).min())/2))/2).shift(i+15)-\
    ((df.loc[masky, 'High'].rolling(i+43, min_periods=1).max()+df.loc[masky, 'Low'].rolling(i+43, min_periods=1).min())/2).shift(i+15)
    
        
    return df

def tech_features(df, masks, nlags= [2, 3, 5, 7]):
    for j in nlags:
#         df['bolinger_band_low_' + str(j) + 'd'] = 0
#         df['bolinger_band_high_' + str(j) + 'd'] = 0
#         df['adx_' + str(j) + 'd'] = 0
#         df['adx_neg_' + str(j) + 'd'] = 0
#         df['adx_pos_' + str(j) + 'd'] = 0
#         df['aroon_down_' + str(j) + 'd'] = 0
#         df['aroon_up_' + str(j) + 'd'] = 0
#         df['ema_indicator_' + str(j) + 'd'] = 0
#         df['macd_indicator_' + str(j) + 'd'] = 0
        
        
        for mask in masks:
            masky = (df['ticker'] == mask)
            
            #Nikitos features
            
            df.loc[masky, 'close_ma_'+str(j)] = df.loc[masky, 'Close']-(df.loc[masky, 'Close'].rolling(j, min_periods=1).mean())
            df.loc[masky, 'close_ema_'+str(j)] = df.loc[masky, 'Close']-(df['Close'].ewm(span=j, min_periods=1).mean())
            df.loc[masky, 'close_3ema_'+str(j)] = df.loc[masky, 'Close']-(3*(df.loc[masky, 'Close'].ewm(span=j, min_periods=1).mean())
                                                      -3*(df.loc[masky, 'Close'].ewm(span=j, min_periods=1).mean()).ewm(span=j, min_periods=1).mean()+
                                                      ((df.loc[masky, 'Close'].ewm(span=j, min_periods=1).mean()).ewm(span=j, min_periods=1).mean()).ewm(span=j, min_periods=1).mean())
            
            
            df.loc[masky, '%b_'+str(j)] = (df.loc[masky, 'Close']-
                                           (df.loc[masky, 'Close'].rolling(j, min_periods=1).mean()-
                                            2*(df.loc[masky, 'Close'].rolling(j, min_periods=1).std())))/((df.loc[masky, 'Close'].rolling(j, min_periods=1).mean()
                                                                                                                                                  +2*(df.loc[masky, 'Close'].rolling(j, min_periods=1).std()))-
                                                                                                                                                 (df.loc[masky, 'Close'].rolling(j, min_periods=1).mean()-
                                                                                                                                                  2*(df.loc[masky, 'Close'].rolling(j, min_periods=1).std())))
            
            df.loc[masky, 'bandwidth_'+str(j)] = ((df.loc[masky, 'Close'].rolling(j, min_periods=1).mean()+2*(df.loc[masky, 'Close'].rolling(j, min_periods=1).std()))-
                                        (df.loc[masky, 'Close'].rolling(j, min_periods=1).mean()-2*(df.loc[masky, 'Close'].rolling(j, min_periods=1).std())))/(df.loc[masky, 'Close'].rolling(j, min_periods=1).mean()-2*(df.loc[masky, 'Close'].rolling(j, min_periods=1).std()))
            
            
            
            df.loc[masky, 'bull_'+str(j)]= df.loc[masky, 'High']-(df.loc[masky, 'Close'].ewm(span=j, min_periods=1).mean())
            
            df.loc[masky, 'bear_'+str(j)]= df.loc[masky, 'Low']-(df.loc[masky, 'Close'].ewm(span=j, min_periods=1).mean())
            
            df.loc[masky, 'roc_'+str(j)]=((df.loc[masky, 'Close']-df.loc[masky, 'Close'].shift(j))/df.loc[masky, 'Close'].shift(j))
            
            
            #My features
            
#             df.loc[masky, 'MA_' + str(j) + 'd'] = df['Close'].rolling(window=j).mean()
            
#             df.loc[masky, 'adx_' + str(j) + 'd'] = adx(df.loc[masky, 'High'],df.loc[masky, 'Low'],
#                                                                      df.loc[masky, 'Close'], n=j, fillna=False)
            
#             df.loc[masky, 'adx_neg_' + str(j) + 'd'] = adx_neg(df.loc[masky, 'High'],df.loc[masky, 'Low'],
#                                                                      df.loc[masky, 'Close'], n=j, fillna=False)
#             df.loc[masky, 'adx_pos_' + str(j) + 'd'] = adx_pos(df.loc[masky, 'High'],df.loc[masky, 'Low'],
#                                                                      df.loc[masky, 'Close'],
#                                                                      n=j, fillna=True)
        
#             df.loc[masky, 'aroon_down_' + str(j) + 'd'] = aroon_down(df.loc[masky, 'Close'], n=j, fillna = True)
#             df.loc[masky, 'aroon_up_' + str(j) + 'd'] = aroon_up(df.loc[masky, 'Close'], n=j, fillna = True)
            
#             #do something with macd periods
#             #make up features with MACD
            
#             df.loc[masky, 'RSI_' + str(j) + 'd'] = rsi(close, n=j, fillna=False)
#             df.loc[masky, 'ema_indicator_' + str(j) + 'd'] = ema_indicator(df.loc[masky, 'Close'], n=j, fillna=True)
            
    return df



def create_lags(df, masks, nlags = [1,2,5,7]):
    for j in nlags:
        for mask in masks:
            masky = (df['ticker'] == mask)
            df.loc[masky, 'lag_Close_' + str(j) + 'd'] = df.loc[masky, 'Close'].shift(j)
            df.loc[masky, 'lag_Open_' + str(j) + 'd'] = df.loc[masky, 'Open'].shift(j)
            
        
    return(df)

In [32]:
%%time 
df = tech_features(df, masks)

Wall time: 17min 10s


In [33]:
%%time
df = not_classical(df, masks)

Wall time: 5min 10s


In [34]:
df.head()

,index,ticker,Date,Adj Close,Close,High,Low,Open,Volume,Name,...,tenkan_a_12,tenkan_b_12,kijun_a_12,kijun_b_12,a_b_12,tenkan_a_15,tenkan_b_15,kijun_a_15,kijun_b_15,a_b_15
0,0,AAL,2008-12-31,7.405228,7.73,7.87,7.48,7.48,4194100.0,American Airlines Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AAL,2009-01-02,8.037499,8.39,8.48,7.67,7.73,5167000.0,American Airlines Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AAL,2009-01-05,7.980019,8.33,8.39,7.96,8.38,3457100.0,American Airlines Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,AAL,2009-01-06,8.679349,9.06,9.21,8.13,8.15,5731000.0,American Airlines Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,AAL,2009-01-07,8.698509,9.08,9.47,8.66,8.66,5468900.0,American Airlines Group,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(1168477, 82)